In [ ]:
import pandas as pd
import numpy as np

# Завантаження даних про пари та їх частоти
pair_counts = pd.read_csv('unique_pair_counts.csv')

# Припустимо, що в нас є файл з ембедінгами продуктів (product_embeddings.csv)
product_embeddings = pd.read_csv('/Users/ivanna/Documents/University/semester_5/PROJECT/new_data/products_final_embedding.csv')

# Приклад: product_id, embedding_1, embedding_2, ..., embedding_n
# Перевірка структури ембедінгів
print(product_embeddings.head())

In [ ]:
import pandas as pd
import numpy as np

# Завантаження даних з CSV
embeddings = pd.read_csv('/Users/ivanna/Documents/University/semester_5/PROJECT/new_data/products_final_embedding.csv')

# Функція для розпарсення рядка у список чисел
def parse_embedding(embedding_str):
    # Видаляємо зайві символи та перетворюємо на список чисел
    embedding_str = embedding_str.strip("[]")  # Видаляємо квадратні дужки
    return np.fromstring(embedding_str, sep=' ')

# Застосовуємо функцію до колонки 'embedding'
embeddings['embedding_parsed'] = embeddings['embedding'].apply(parse_embedding)

# Розбиваємо кожен масив на окремі колонки
embedding_df = pd.DataFrame(embeddings['embedding_parsed'].tolist(), index=embeddings.index)

# Об'єднуємо з основною таблицею
embeddings = pd.concat([embeddings, embedding_df], axis=1)

# Видаляємо стару колонку з рядковими ембедінгами
embeddings = embeddings.drop(['embedding', 'embedding_parsed'], axis=1)

# Перевіряємо результат
print(embeddings.head())

# Зберігаємо очищений файл
embeddings.to_csv('cleaned_embeddings.csv', index=False)


In [2]:
import pandas as pd

# Завантажуємо файл з ембедінгами
embeddings = pd.read_csv('cleaned_embeddings.csv')

# Перевіряємо на наявність NaN значень
print(embeddings.isna().sum())

# Переконуємось, що всі ембедінги числові
for col in embeddings.columns[4:]:  # Пропускаємо перші колонки, залишаємо лише ембедінги
    try:
        pd.to_numeric(embeddings[col], errors='raise')
    except Exception as e:
        print(f"Некоректні дані в колонці {col}: {e}")

Unnamed: 0       0
product_id       0
product_name     0
aisle_id         0
department_id    0
                ..
763              0
764              0
765              0
766              0
767              0
Length: 773, dtype: int64


In [ ]:
import pandas as pd
import numpy as np

# Завантаження даних про пари та ембедінги
pair_counts = pd.read_csv('unique_pair_counts.csv')
product_embeddings = pd.read_csv('/Users/ivanna/Documents/University/semester_5/PROJECT/new_data/products_final_embedding.csv')

# Перевіряємо та конвертуємо ембедінги в числовий формат
for col in product_embeddings.columns[1:]:  # Пропускаємо 'product_id'
    product_embeddings[col] = pd.to_numeric(product_embeddings[col], errors='coerce').fillna(0.0)

# Перетворюємо ембедінги на словник для швидкого доступу
embeddings_dict = product_embeddings.set_index('product_id').to_dict(orient='index')

def aggregate_embeddings(row):
    a, b = row['product_1'], row['product_2']
    prob = row['probability']

    # Отримуємо ембедінги продуктів або вектор нулів, якщо продукт відсутній
    emb_a = np.array(list(embeddings_dict.get(a, np.zeros(10)).values()), dtype=np.float64)
    emb_b = np.array(list(embeddings_dict.get(b, np.zeros(10)).values()), dtype=np.float64)

    # Агрегуємо ембедінги з урахуванням ймовірностей
    aggregated_embedding = prob * (emb_a + emb_b) / 2
    return aggregated_embedding

# 1. Підраховуємо кількість появ кожного продукту
all_products = pd.concat([pair_counts['product_1'], pair_counts['product_2']])
product_frequencies = all_products.value_counts().reset_index()
product_frequencies.columns = ['product_id', 'count']

# Перетворюємо частоти продуктів у словник для швидкого доступу
product_count_dict = product_frequencies.set_index('product_id')['count'].to_dict()

# 2. Обчислюємо ймовірність для кожної пари
def calculate_probability(row):
    a, b, count_ab = row['product_1'], row['product_2'], row['count']
    count_a = product_count_dict.get(a, 1)  # Якщо продукту немає, використовуємо 1 як значення
    count_b = product_count_dict.get(b, 1)
    probability = count_ab / (count_a + count_b - count_ab)
    return probability

pair_counts['probability'] = pair_counts.apply(calculate_probability, axis=1)

# 3. Агрегуємо ембедінги для кожної пари
pair_counts['aggregated_embedding'] = pair_counts.apply(aggregate_embeddings, axis=1)

# 4. Зберігаємо результати
pair_counts.to_csv('aggregated_pair_embeddings2.csv', index=False)

print("Агрегація ембедінгів завершена! Результати збережено в 'aggregated_pair_embeddings.csv'.")
